In [1]:
# Import our dependencies
import pandas as pd
from path import Path
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
# Keras
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense

C:\Users\Seva\anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Seva\anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Seva\anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Seva\anaconda3\envs\PythonData\lib\site-packages\ten

In [2]:
# Import our clean dataset
file_path = Path("../clean_data/combined_olympic_data.csv")
df = pd.read_csv(file_path)
df.head(10)

,country_name,population,gdp_per_capita,human_development_index,gender_inequality_index,corruption_perceptions_index,total
0,Afghanistan,33370794,613.856689,0.465,0.676,12,1
1,Albania,2889104,4578.631994,0.733,0.267,33,0
2,United Arab Emirates,9214175,43751.838890,0.835,0.247,70,0
3,Argentina,42669500,12334.798250,0.836,0.364,34,4
4,Armenia,2912403,3986.231624,0.733,0.299,37,2
5,Australia,23475686,62510.791170,0.935,0.123,80,38
6,Austria,8546356,51717.495940,0.885,0.083,72,17
7,Azerbaijan,9535079,7891.313147,0.751,0.330,29,9
8,Burundi,9844297,274.857948,0.400,0.483,20,0
9,Belgium,11209057,47700.540360,0.890,0.076,76,3


In [3]:
df["total"].value_counts()

0      75
1      18
2       8
3       6
4       5
6       5
8       4
17      3
5       3
9       3
12      3
13      3
7       2
15      2
18      2
38      2
100     1
19      1
16      1
21      1
23      1
31      1
36      1
43      1
44      1
46      1
50      1
63      1
70      1
97      1
132     1
Name: total, dtype: int64

In [4]:
bin_labels = ['0', '1-5', '6-10', '11-30', '31-50', '51+']
num_bins = len(bin_labels)

df['medal_grouping'] = pd.cut(df['total'],
                              bins=[-1, 0, 5, 10, 30, 50, 140],
                              labels=bin_labels)
df['numerical_medal_grouping'] = pd.cut(df['total'],
                              bins=[-1, 0, 5, 10, 30, 50, 140],
                              labels=range(num_bins))

In [5]:
df['medal_grouping'].value_counts()

0        75
1-5      40
11-30    17
6-10     14
31-50     8
51+       5
Name: medal_grouping, dtype: int64

In [6]:
df = df.set_index("country_name",drop=True)
new_df = df.rename(columns={"total": "count_of_medals"})
new_df.head()

,population,gdp_per_capita,human_development_index,gender_inequality_index,corruption_perceptions_index,count_of_medals,medal_grouping,numerical_medal_grouping
country_name,,,,,,,,
Afghanistan,33370794,613.856689,0.465,0.676,12,1,1-5,1
Albania,2889104,4578.631994,0.733,0.267,33,0,0,0
United Arab Emirates,9214175,43751.838890,0.835,0.247,70,0,0,0
Argentina,42669500,12334.798250,0.836,0.364,34,4,1-5,1
Armenia,2912403,3986.231624,0.733,0.299,37,2,1-5,1


### Split our preprocessed data into our features and target arrays

In [7]:
# Create our target
y = new_df["numerical_medal_grouping"].values
y[:20]

[1, 0, 0, 1, 1, ..., 1, 0, 3, 0, 3]
Length: 20
Categories (6, int64): [0 < 1 < 2 < 3 < 4 < 5]

In [8]:
# Optimizing and transforming features
# Transforming CPI to values between 0-1
new_df["corruption_perceptions_index"] = new_df["corruption_perceptions_index"] / 100
# Transforming GII to invert values. Higher values will now correspond to more gender equality.
new_df["gender_inequality_index"] = 1 - new_df["gender_inequality_index"]
# Replacing GDP per capita with total GDP
new_df["gdp_total"] = new_df["population"] * new_df["gdp_per_capita"]
# X = new_df.drop(["count_of_medals","population","gdp_per_capita", "human_development_index", "gender_inequality_index", "corruption_perceptions_index"], axis=1)
new_df.head()

,population,gdp_per_capita,human_development_index,gender_inequality_index,corruption_perceptions_index,count_of_medals,medal_grouping,numerical_medal_grouping,gdp_total
country_name,,,,,,,,,
Afghanistan,33370794,613.856689,0.465,0.324,0.12,1,1-5,1,2.048489e+10
Albania,2889104,4578.631994,0.733,0.733,0.33,0,0,0,1.322814e+10
United Arab Emirates,9214175,43751.838890,0.835,0.753,0.70,0,0,0,4.031371e+11
Argentina,42669500,12334.798250,0.836,0.636,0.34,4,1-5,1,5.263197e+11
Armenia,2912403,3986.231624,0.733,0.701,0.37,2,1-5,1,1.160951e+10


In [9]:
# Create our features
# X = new_df.drop(["count_of_medals", "medal_grouping", "numerical_medal_grouping","gdp_per_capita", "population"], axis=1).values
X = new_df.drop(["count_of_medals", "medal_grouping", "numerical_medal_grouping"], axis=1).values

X

array([[3.33707940e+07, 6.13856689e+02, 4.65000000e-01, 3.24000000e-01,
        1.20000000e-01, 2.04848851e+10],
       [2.88910400e+06, 4.57863199e+03, 7.33000000e-01, 7.33000000e-01,
        3.30000000e-01, 1.32281440e+10],
       [9.21417500e+06, 4.37518389e+04, 8.35000000e-01, 7.53000000e-01,
        7.00000000e-01, 4.03137100e+11],
       [4.26695000e+07, 1.23347982e+04, 8.36000000e-01, 6.36000000e-01,
        3.40000000e-01, 5.26319674e+11],
       [2.91240300e+06, 3.98623162e+03, 7.33000000e-01, 7.01000000e-01,
        3.70000000e-01, 1.16095129e+10],
       [2.34756860e+07, 6.25107912e+04, 9.35000000e-01, 8.77000000e-01,
        8.00000000e-01, 1.46748371e+12],
       [8.54635600e+06, 5.17174959e+04, 8.85000000e-01, 9.17000000e-01,
        7.20000000e-01, 4.41996132e+11],
       [9.53507900e+06, 7.89131315e+03, 7.51000000e-01, 6.70000000e-01,
        2.90000000e-01, 7.52442943e+10],
       [9.84429700e+06, 2.74857948e+02, 4.00000000e-01, 5.17000000e-01,
        2.00000000e-01, 

In [10]:
# Splitting data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=15)

In [11]:
#Create a StandardScaler instances
scaler = MinMaxScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [12]:
# Saving values for result comparison later on
y_test_values = y_test

In [13]:
# We need to convert our target labels (expected values) to categorical data
num_classes = num_bins
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)
# Original label of `1` is one-hot encoded as `0100000000`
y_train[0]

array([0., 1., 0., 0., 0., 0.], dtype=float32)

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

number_input_features = len(X_train[0])
hidden_nodes_layer1 = 3 * number_input_features
hidden_nodes_layer2 = 2 * number_input_features
hidden_nodes_layer3 = 2 * number_input_features

nn = Sequential()
number_input_features

6

In [15]:
# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=num_classes, activation="softmax"))
# After bucketing the medal count, you'd have to change this to have mulitiple outputs. 
# In other words, you'd need one node for each bucket. And then you'd change the activation to "softmax."
# You'd also have to one-hot encode your y values. 

# Check the structure of the model
nn.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 18)                126       
_________________________________________________________________
dense_1 (Dense)              (None, 12)                228       
_________________________________________________________________
dense_2 (Dense)              (None, 12)                156       
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 78        
Total params: 588
Trainable params: 588
Non-trainable params: 0
_________________________________________________________________


In [16]:
# # Import checkpoint dependencies
# import os
# from tensorflow.keras.callbacks import ModelCheckpoint

# # Define the checkpoint path and filenames
# os.makedirs("checkpoints/",exist_ok=True)
# checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [17]:
# Compile the model
nn.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])

In [18]:
# # Create a callback that saves the model's weights every 5 epochs
# cp_callback = ModelCheckpoint(
#     filepath=checkpoint_path,
#     verbose=1,
#     save_weights_only=True,
#     save_freq=1000)

In [19]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=150, shuffle=True)

Epoch 1/150
119/119 [==============================] - 0s 595us/sample - loss: 1.7960 - acc: 0.1008
Epoch 2/150
119/119 [==============================] - 0s 34us/sample - loss: 1.7827 - acc: 0.3866
Epoch 3/150
119/119 [==============================] - 0s 42us/sample - loss: 1.7712 - acc: 0.4454
Epoch 4/150
119/119 [==============================] - 0s 42us/sample - loss: 1.7616 - acc: 0.4538
Epoch 5/150
119/119 [==============================] - 0s 34us/sample - loss: 1.7521 - acc: 0.4538
Epoch 6/150
119/119 [==============================] - 0s 34us/sample - loss: 1.7417 - acc: 0.4538
Epoch 7/150
119/119 [==============================] - 0s 35us/sample - loss: 1.7329 - acc: 0.4538
Epoch 8/150
119/119 [==============================] - 0s 33us/sample - loss: 1.7230 - acc: 0.4538
Epoch 9/150
119/119 [==============================] - 0s 25us/sample - loss: 1.7126 - acc: 0.4538
Epoch 10/150
119/119 [==============================] - 0s 25us/sample - loss: 1.7016 - acc: 0.4538
Epoch 11

119/119 [==============================] - 0s 34us/sample - loss: 1.1660 - acc: 0.5294
Epoch 83/150
119/119 [==============================] - 0s 25us/sample - loss: 1.1620 - acc: 0.5294
Epoch 84/150
119/119 [==============================] - 0s 25us/sample - loss: 1.1582 - acc: 0.5294
Epoch 85/150
119/119 [==============================] - 0s 25us/sample - loss: 1.1566 - acc: 0.5378
Epoch 86/150
119/119 [==============================] - 0s 34us/sample - loss: 1.1524 - acc: 0.5294
Epoch 87/150
119/119 [==============================] - 0s 25us/sample - loss: 1.1478 - acc: 0.5378
Epoch 88/150
119/119 [==============================] - 0s 25us/sample - loss: 1.1447 - acc: 0.5378
Epoch 89/150
119/119 [==============================] - 0s 25us/sample - loss: 1.1412 - acc: 0.5378
Epoch 90/150
119/119 [==============================] - 0s 25us/sample - loss: 1.1379 - acc: 0.5378
Epoch 91/150
119/119 [==============================] - 0s 25us/sample - loss: 1.1356 - acc: 0.5462
Epoch 92/150


In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

40/40 - 0s - loss: 1.1313 - acc: 0.5750
Loss: 1.1313021063804627, Accuracy: 0.574999988079071


In [21]:
# Make predictions with scaled test data
y_test_pred = nn.predict(X_test_scaled)

In [22]:
y_test

array([[1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [1.

In [23]:
y_test_pred.round(1)

array([[0.8, 0.2, 0. , 0. , 0. , 0. ],
       [0.5, 0.3, 0.1, 0.1, 0. , 0. ],
       [0.5, 0.3, 0.1, 0.1, 0. , 0. ],
       [0.8, 0.2, 0. , 0. , 0. , 0. ],
       [0.2, 0.4, 0.2, 0.2, 0.1, 0. ],
       [0.7, 0.2, 0. , 0. , 0. , 0. ],
       [0.8, 0.2, 0. , 0. , 0. , 0. ],
       [0.4, 0.4, 0.1, 0.1, 0. , 0. ],
       [0.5, 0.3, 0.1, 0.1, 0. , 0. ],
       [0. , 0.2, 0.2, 0.2, 0.3, 0.1],
       [0.1, 0.3, 0.2, 0.2, 0.1, 0. ],
       [0.6, 0.3, 0. , 0. , 0. , 0. ],
       [0.2, 0.4, 0.1, 0.2, 0. , 0. ],
       [0.5, 0.3, 0.1, 0.1, 0. , 0. ],
       [0.4, 0.4, 0.1, 0.1, 0. , 0. ],
       [0.5, 0.4, 0.1, 0.1, 0. , 0. ],
       [0.7, 0.2, 0. , 0. , 0. , 0. ],
       [0.3, 0.4, 0.1, 0.2, 0. , 0. ],
       [0.5, 0.3, 0.1, 0.1, 0. , 0. ],
       [0.1, 0.3, 0.2, 0.3, 0.2, 0. ],
       [0. , 0. , 0. , 0. , 0.2, 0.7],
       [0.1, 0.3, 0.2, 0.2, 0.2, 0. ],
       [0.8, 0.2, 0. , 0. , 0. , 0. ],
       [0.2, 0.4, 0.2, 0.2, 0. , 0. ],
       [0.6, 0.3, 0.1, 0. , 0. , 0. ],
       [0.6, 0.3, 0.1, 0.

In [24]:
nn.predict_classes(X_test_scaled)

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 4, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 5, 1,
       0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 4, 4, 0, 4], dtype=int64)

### Saving the model

In [25]:
# Export our model to HDF5 file
nn.save("DL_trained_categories_model.h5")

### Creating CSV with model results

In [26]:
# Replicating training and testing dataset split but with country names 
X_train, X_test, y_train_countries, y_test_countries = train_test_split(X, new_df.index.values, test_size=0.25, random_state=15)

In [27]:
y_test_countries

array(['Zambia', 'Iran', 'Algeria', 'Congo Republic', 'Portugal',
       'Cambodia', 'Sao Tome and Principe', 'Trinidad and Tobago', 'Peru',
       'Switzerland', 'Belgium', 'Jamaica', 'Libya', 'Philippines',
       'Chile', 'Mongolia', 'Gabon', 'Malta', 'South Africa', 'Austria',
       'India', 'Singapore', 'Nepal', 'Virgin Islands (U.S.)', 'Pakistan',
       'Sri Lanka', 'Hungary', 'Democratic Republic of the Congo',
       'Lebanon', 'Kazakhstan', 'Mali', 'Tanzania', 'Haiti', 'Croatia',
       'Chad', 'Guyana', 'France', 'Australia', 'Panama', 'Russia'],
      dtype=object)

In [28]:
# Creating DF of model result countries
results_df = pd.DataFrame(y_test_countries)
results_df['count_of_medals'] = y_test_values
results_df['model_results'] = nn.predict_classes(X_test_scaled)
results_df = results_df.rename(columns={results_df.columns[0]: "count_of_medals"})

results_df

,count_of_medals,count_of_medals,model_results
0,Zambia,0,0
1,Iran,3,0
2,Algeria,1,0
3,Congo Republic,0,0
4,Portugal,1,1
5,Cambodia,0,0
6,Sao Tome and Principe,0,0
7,Trinidad and Tobago,1,0
8,Peru,0,0
9,Switzerland,3,4


In [29]:
# Writing model results to csv
results_df.to_csv("../clean_data/model_results/neural_network_model_results.csv", index=False)